# Mini-batch

```text
- Variation of gradient descent that splits the training data into smalleer batches that are used for calculating the model error and updating the model weights.
- Batch size is often powers of 2. e.g. between 2 to 512.
- Training in batches can reduce the computational time because of vectorization (matrix multiplication instead of for-loops).
- It smoothens learning by averaging the loss over many samples thereby reducing overfitting.
```